In [1]:
from sklearn.preprocessing import MinMaxScaler
import csv
import pandas as pd
import numpy as np
import os
import pickle
import itertools
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import random
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error as MSLE
random.seed(123)
plt.rcParams["font.sans-serif"]=["SimHei"]
from datetime import datetime
pd.set_option("display.max_rows", 1000)    #設定最大能顯示1000rows
pd.set_option("display.max_columns", 1000) #設定最大能顯示1000columns
path = r"C:\Users\user\Desktop\ETC"

In [2]:
acc_1_fin=pd.read_csv(path+r"/acc1_fin_dummy_and_sd.csv")
print(acc_1_fin.columns)
print(acc_1_fin.shape)

Index(['Unnamed: 0', '回堵情況', '處理時間', '總通行量', '總平均車速', '里程數', '速限', '24小時內死亡人數',
       '2-30日內死亡人數', '受傷人數',
       ...
       '肇因研判(子)_44', '肇因研判(子)_48', '肇因研判(子)_49', '肇因研判(子)_50', '肇因研判(子)_65',
       '肇因研判(子)_67', '是否為連續假日_1', '是否為假日_1', '是否開放路肩_1', '通報級數'],
      dtype='object', length=326)
(7242, 326)


In [3]:
with open(file=path+r'\variable_select_varname_by_xgb.lst',mode="rb") as inhis :  
    variable_select_varname = pickle.load(file=inhis)   

In [4]:
acc_1_dim_reduction=acc_1_fin[variable_select_varname]

In [5]:
acc_1_dim_reduction.columns

Index(['總通行量', '總平均車速', '里程數', '速限', '受傷人數', '事故涉及車輛數', '通報地區_北控', '通報地區_南控',
       '向車道_南', '通報來源_2', '通報來源_3', '通報來源_4', '通報來源_5', '縣市_新北市', '縣市_桃園市',
       '縣市_臺中市', '縣市_臺北市', '縣市_苗栗縣', '縣市_雲林縣', '縣市_高雄市', '市區鄉鎮_中山區',
       '市區鄉鎮_五股區', '市區鄉鎮_內湖區', '市區鄉鎮_大雅區', '市區鄉鎮_新市區', '市區鄉鎮_林口區', '市區鄉鎮_汐止區',
       '市區鄉鎮_泰山區', '市區鄉鎮_蘆竹區', '當事者車種(子)_A31', '當事者車種(子)_B03', '當事者車種(子)_B12',
       '月_4', '月_5', '月_6', '月_9', '月_10', '月_12', '星期_1', '星期_2', '星期_3',
       '星期_4', '星期_5', '星期_6', '小時_4', '小時_7', '小時_9', '小時_11', '小時_14',
       '小時_15', '小時_16', '小時_18', '小時_23', '天候代碼_6', '天候代碼_8', '光線代碼_3',
       '光線代碼_4', '道路型態(子)_10', '道路型態(子)_14', '路面狀況-狀態_4', '車道劃分設施-分向設施(主)_5',
       '車道劃分設施-分向設施(子)_10', '車道劃分設施-分道設施-快車道或一般車道間_3', '事故類型及型態(主)_3',
       '事故類型及型態(子)_12', '事故類型及型態(子)_13', '事故類型及型態(子)_20', '當事者車種(主)_1',
       '當事者車種(主)_4', '當事者車種(主)_6', '當事者車種(主)_7', '肇因研判(主)_4', '肇因研判(主)_6',
       '肇因研判(子)_7', '肇因研判(子)_14', '肇因研判(子)_16', '肇因研判(子)_23', '是否為連續假日_1',
       '是否為假日_1', '通報級數'],


In [6]:
X1 = acc_1_dim_reduction
y1 = acc_1_fin["處理時間"]

In [7]:
X1.columns

Index(['總通行量', '總平均車速', '里程數', '速限', '受傷人數', '事故涉及車輛數', '通報地區_北控', '通報地區_南控',
       '向車道_南', '通報來源_2', '通報來源_3', '通報來源_4', '通報來源_5', '縣市_新北市', '縣市_桃園市',
       '縣市_臺中市', '縣市_臺北市', '縣市_苗栗縣', '縣市_雲林縣', '縣市_高雄市', '市區鄉鎮_中山區',
       '市區鄉鎮_五股區', '市區鄉鎮_內湖區', '市區鄉鎮_大雅區', '市區鄉鎮_新市區', '市區鄉鎮_林口區', '市區鄉鎮_汐止區',
       '市區鄉鎮_泰山區', '市區鄉鎮_蘆竹區', '當事者車種(子)_A31', '當事者車種(子)_B03', '當事者車種(子)_B12',
       '月_4', '月_5', '月_6', '月_9', '月_10', '月_12', '星期_1', '星期_2', '星期_3',
       '星期_4', '星期_5', '星期_6', '小時_4', '小時_7', '小時_9', '小時_11', '小時_14',
       '小時_15', '小時_16', '小時_18', '小時_23', '天候代碼_6', '天候代碼_8', '光線代碼_3',
       '光線代碼_4', '道路型態(子)_10', '道路型態(子)_14', '路面狀況-狀態_4', '車道劃分設施-分向設施(主)_5',
       '車道劃分設施-分向設施(子)_10', '車道劃分設施-分道設施-快車道或一般車道間_3', '事故類型及型態(主)_3',
       '事故類型及型態(子)_12', '事故類型及型態(子)_13', '事故類型及型態(子)_20', '當事者車種(主)_1',
       '當事者車種(主)_4', '當事者車種(主)_6', '當事者車種(主)_7', '肇因研判(主)_4', '肇因研判(主)_6',
       '肇因研判(子)_7', '肇因研判(子)_14', '肇因研判(子)_16', '肇因研判(子)_23', '是否為連續假日_1',
       '是否為假日_1', '通報級數'],


In [8]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2) # 0.2 test_size means 20%

In [9]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

# MLP

In [10]:
# MLP是多层感知器，使用的是前馈神经网络
#只支持交叉熵损失，使用mlp.predict_proda()，最小化交叉熵，同时给似然估计p(y|x)

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [11]:
mlp_gsc = GridSearchCV(MLPRegressor(activation='relu', solver='adam', batch_size='auto',
    learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=5000, shuffle=True,
    random_state=1, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False), param_grid={
                  'hidden_layer_sizes': [x for x in itertools.product((10,20,30,40,50,100),repeat=3)]},
              scoring='neg_mean_squared_error', verbose=0,n_jobs=-1, cv=10, refit=True)

mlp_gsc.fit(X1_train,y1_train)
mlp_gsc.best_params_

{'hidden_layer_sizes': (20, 100, 100)}

In [12]:
mlp = MLPRegressor(random_state=0, activation='relu', solver='adam',
                   hidden_layer_sizes=mlp_gsc.best_params_["hidden_layer_sizes"])
mlp_scores = cross_val_score(mlp, X1_train, y1_train, cv=10,scoring='neg_mean_squared_error')

C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [13]:
mlp_fit = mlp.fit(X1_train,y1_train)

In [14]:
## predict_y
mlp_1_pred=mlp.predict(X1_test)
mlp_1_pred=mlp_1_pred.flatten()

In [15]:
# rmse
print(np.sqrt(mean_squared_error(y1_test,mlp_1_pred)))

16.260993433718514


In [16]:
from sklearn.metrics import mean_absolute_error

In [17]:
# mae
print(mean_absolute_error(y1_test,mlp_1_pred))

9.797419287882319


In [ ]:
var_imp=X1_train.columns[xgb9.feature_importances_>0.007]
xgb9.feature_importances_[np.where(xgb9.feature_importances_>0.007)]

In [ ]:
# 係數視覺化
y_importances = xgb9.feature_importances_[np.where(xgb9.feature_importances_>0.007)]
x_importances = var_imp
y_pos = np.arange(len(x_importances))
plt.figure(figsize=(10,10))
plt.barh(y_pos, y_importances, align='center')
plt.yticks(y_pos, x_importances,fontsize=14)
plt.xlabel('Importances')
plt.xlim(0,0.3)
plt.title('Features Importances',fontsize=14)
plt.savefig('XGB_select(OBJ).png',bbox_inches='tight')
plt.show()